# Stream Ciphers

One-time pad limitations: the same key can't be use twice.

**Proof**

$$
c_0 = k \oplus m_0 \\ 
c_1 = k \oplus m_1 \\ 
c_0 \oplus c_1 \\
(k \oplus m_0) \oplus (k \oplus m_1) \\
m_0 \oplus m_1 \\
$$

If we have $m_0$ y $m_1$, we can get $m_1$

$$
m_0 \oplus m_1 \oplus m_0 = m_1
$$

**pseudorandom generator**

Stream Ciphers use a pseudorandom generator that generates key from a seed

$$
G(s_0) \rightarrow h \\
si s_1 = s_0 entonces \\
G(s_1) \rightarrow h
$$

$\mathcal{E} = (E, D)$, $\mathcal{K}:=\{0, 1\}^l$, $\mathcal{M}:=\mathcal{C}:=\{0, 1\}^{\leq L}$ con $l < L$

$$
E(k, m) = G(k)[0, ..., v-1] \oplus m = c \\
v = |m| \\
\\
D(k, c) = G(k)[0, ..., v-1] \oplus c = m \\
v = |c|
$$

Salsa is a fast stream cipher ans Chacha a variant. They both use two main components: a padding function pad(s, j, n) that combines a 256-bit seed s with a 64-bit counter j and 32-64 nonce n to form a 512-bit block. It also have a fixed public permutation $pi$ (input 512bits and output 512 bits). The algorithm generates $L < 2^{64}$ psudorandom block of 512-bits

~~~
G(s, n)
    for j = 0, L - 1, 1
        h <- pad(s, j, 0)
        rj <- pi(h) xor h
    return r

~~~

In [1]:
from typing import List, Tuple

### Words

A word is an elment of 32 bits. THe sum of two word is mod $2^32$

In [7]:
# Notation: + is the addition mod 2^32

def sum32(a: int, b: int) -> int:
    # Suma mod 2^32
    return (a + b) & ((1 << 32) - 1)

assert(sum32(2**32, 3) == 3), "Error"

In [8]:

def text_to_int(string: str) -> int:
    # I copy this from StackOverFlow
    nchars = len(string)
    return sum(ord(string[byte])<<8*(nchars-byte-1) for byte in range(nchars))

assert(text_to_int("expa"[::-1]) == 0x61707865), "Error"

In [9]:

def int_to_text(integer: int) -> str:
    # I dont remmber where I found this
    result = ""
    while integer > 0:
        byte = integer & 0xFF
        result = chr(byte) + result
        integer >>= 8
    
    return result

assert("expa" == int_to_text(0x61707865)[::-1]), "Error"

In [10]:

# Chatgpt assited generated code 
# Do no ask me how it works

def to_words(value: int,
             word_size: int,
             words_num: int) -> List[int]:
    
    hex_string = hex(value)[2:]  # Remove '0x' prefix
    
    binary_string = bin(int(hex_string, 16))[2:].zfill(len(hex_string) * 4)  # Convert to binary
    padding_bits = word_size * words_num - len(binary_string)
    
    if padding_bits < 0:
        raise ValueError("Number of words specified is insufficient for the given value.")
    
    binary_string = binary_string.zfill(word_size * words_num)  # Add leading zeros for padding
    
    words = [int(binary_string[i:i+word_size], 2) for i in range(0, len(binary_string), word_size)]
    
    return words

assert(to_words(0x112233445566778899aabbccddeeff00, 32, 8) == [0x0, 0x0, 0x0, 0x0, 0x11223344, 0x55667788, 0x99aabbcc, 0xddeeff00]), "Error"

In [11]:

def from_words(words: List[int], word_size: int) -> int:
    binary_string = ''.join(format(word, f'0{word_size}b') for word in words)
    hex_string = hex(int(binary_string, 2))
    return int(hex_string, 16)

assert(0x112233445566778899aabbccddeeff00 == from_words([0x0, 0x0, 0x0, 0x0, 0x11223344, 0x55667788, 0x99aabbcc, 0xddeeff00], 32)), "Error"


## Salsa20

"The core of Salsa20 is a hash function with 64-byte input and 64-byte output. The hash function is used in counter mode as a stream cipher: Salsa20 encrypts a 64-byte block of plaintext by hashing the key, nonce, and block number and xor’ing the result with the plaintext". 

### Pad function

In [28]:
# constant word (32 bits each)
# "Expand 32-byte k"
_c = [text_to_int("expa"[::-1]),
     text_to_int("nd 3"[::-1]),
     text_to_int("2-by"[::-1]),
     text_to_int("te k"[::-1])]

def pad(s: int, j: int, n: int) -> int:
    """A padding function that combies a 256 bits seed s with a 64 bit counter j (j0, j1 of 32 bits) and a b4 bit nonce (n0, n1 of 32 bits) to form a 512 bit block denoted x0, ..., x15 of 32 bits
    
    Args:
        s: Seed (256 bits, 8 words of 32 bits)
        j: counter (64 bits)
        n: Nonce (64 bits)

    Returns:
        512 bits (16 words 32 bits)
    """

    # Convert to words
    s = to_words(s, 32, 8)
    j = to_words(j, 32, 2)
    n = to_words(n, 32, 2)

    return [
        _c[0], s[0], s[1], s[2],
        s[3], _c[1], n[0], n[1],
        j[0], j[1], _c[2], s[4],
        s[5], s[6], s[7], _c[3]
    ]


In [13]:
assert(
    pad(0x112233445566778899aabbccddeeff00,
        0x0123456789abcdef,
        0xfedcba9876543210) == 
        
        [0x61707865, 0x0, 0x0, 0x0, 
         0x0, 0x3320646e, 0xfedcba98, 0x76543210,
         0x1234567, 0x89abcdef, 0x79622d32, 0x11223344,
         0x55667788, 0x99aabbcc, 0xddeeff00, 0x6b206574]
), "Error"

### Public permutation

The public permutation is constructed by iterating a simple permutation a fixed number of times. 

#### Quarterround function

- Input: 4-word sequence 
- Output: 4-word sequence 

The entire function is invertible.

In [14]:
def rot(w: int, r: int) -> int: 
    """Rotate lest for 32 bits

    Args:
        w: word to rotate
        r: I dont remember
    """

    mask = 0xffffffff
    return ((w << r) & mask) | (w >> (32 - r)) 

In [15]:
def QR(a: int, b: int, c: int, d: int) -> Tuple[int, int, int, int]: 
    # Quater round
    b = b ^ rot(sum32(a, d), 7)
    c = c ^ rot(sum32(b, a), 9)
    d = d ^ rot(sum32(c, b), 13)
    a = a ^ rot(sum32(d, c), 18)
    
    return a, b, c, d


In [16]:
# Test cases (from the Salsa20 specification)

assert(QR(0x00000000, 0x00000000, 0x00000000, 0x00000000)
== (0x00000000, 0x00000000, 0x00000000, 0x00000000)), "Faild test 1"
assert(QR(0x00000001, 0x00000000, 0x00000000, 0x00000000)
== (0x08008145, 0x00000080, 0x00010200, 0x20500000)), "Failed test 2"
assert(QR(0x00000000, 0x00000001, 0x00000000, 0x00000000)
== (0x88000100, 0x00000001, 0x00000200, 0x00402000)), "Failed test 3"
assert(QR(0x00000000, 0x00000000, 0x00000001, 0x00000000)
== (0x80040000, 0x00000000, 0x00000001, 0x00002000)), "Failed test 4"
assert(QR(0x00000000, 0x00000000, 0x00000000, 0x00000001)
== (0x00048044, 0x00000080, 0x00010000, 0x20100001)), "Failed test 5"
assert(QR(0xe7e8c006, 0xc4f9417d, 0x6479b4b2, 0x68c67137)
== (0xe876d72b, 0x9361dfd5, 0xf1460244, 0x948541a3)), "Failed test 6"
assert(QR(0xd3917c5b, 0x55f1c407, 0x52a58a7a, 0x8f887a3b)
== (0x3e2f308c, 0xd90a8f36, 0x6ab2a923, 0x2883524c)), "Failed test 7"

#### Round Function

In [17]:
def round(i: List[int]):
    # Doble round
        
    # Odd round (column round)
    i[0], i[4], i[8], i[12] = QR(i[0], i[4], i[8], i[12]) 
    i[5], i[9], i[13], i[1] = QR(i[5], i[9], i[13], i[1]) 
    i[10], i[14], i[2], i[6] = QR(i[10], i[14], i[2], i[6]) 
    i[15], i[3], i[7], i[11] = QR(i[15], i[3], i[7], i[11]) 

    # Even round (row round)
    i[0], i[1], i[2], i[3] = QR(i[0], i[1], i[2], i[3]) 
    i[5], i[6], i[7], i[4] = QR(i[5], i[6], i[7], i[4]) 
    i[10], i[11], i[8], i[9] = QR(i[10], i[11], i[8], i[9])
    i[15], i[12], i[13], i[14] = QR(i[15], i[12], i[13], i[14])

    return i

In [18]:
assert(round([0x00000001, 0x00000000, 0x00000000, 0x00000000,
0x00000000, 0x00000000, 0x00000000, 0x00000000,
0x00000000, 0x00000000, 0x00000000, 0x00000000,
0x00000000, 0x00000000, 0x00000000, 0x00000000]) == [0x8186a22d, 0x0040a284, 0x82479210, 0x06929051,
0x08000090, 0x02402200, 0x00004000, 0x00800000,
0x00010200, 0x20400000, 0x08008104, 0x00000000,
0x20500000, 0xa0000040, 0x0008180a, 0x612a8020]), "Failed test 1"

assert(round([0xde501066, 0x6f9eb8f7, 0xe4fbbd9b, 0x454e3f57,
0xb75540d3, 0x43e93a4c, 0x3a6f2aa0, 0x726d6b36,
0x9243f484, 0x9145d1e8, 0x4fa9d247, 0xdc8dee11,
0x054bf545, 0x254dd653, 0xd9421b6d, 0x67b276c1]) == [0xccaaf672, 0x23d960f7, 0x9153e63a, 0xcd9a60d0,
0x50440492, 0xf07cad19, 0xae344aa0, 0xdf4cfdfc,
0xca531c29, 0x8e7943db, 0xac1680cd, 0xd503ca00,
0xa74b2ad6, 0xbc331c5c, 0x1dda24c7, 0xee928277]), "Failed test 2"

#### The perm function

In [19]:

def perm(x: int, ROUNDS: int = 20): 
    """Permutation 

    Args:
        x: 512 bits (16 words 32 bits)
        ROUNDS: Number of rounds
            - Default 20 (salsa20/20)
            - 8 (salsa20/8)
            - 12 (salsa20/12)
            
    Returns:
        512 bits (16 words 32 bits)
    """
    
    i = x
    
    for _ in range(0, ROUNDS, 2): 
        round(i)
    return i


### Psudorandom generator

In [23]:
L = 1 # 
def G(s: int, n: int):
    r = [0]*L

    for j in range(L):
        h = pad(s, j, n) # 512 bits
        pi = perm(h) # 512 bits

        r[j] = [0]*16
        for i in range(16):
           r[j] = from_words(pi, 32) + from_words(h, 32)
           
    return r

### Example 

In [33]:
k = G(0x112233445566778899aabbccddeeff00, 0x0123456789abcdef)[0]
m = text_to_int("Hello word!")
c = m ^ k
int_to_text(c)

'ôFInòÛ\x84\x16*\x9d\x87³ê\x9c \x0c¢\x19_wëUéf@\x1f¬\x13\x7fÒ\x05Zl¢¡\x89þ/ÿdr\x99PnÅnè\rp\x1fz,\x0bí\x16>\x93öÂÜ\x17OO_'

## ChaCha

In [12]:
def QR_chacha(a, b, c, d):
    a = sum(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum(a, b)
    
    return a, b, c, d

## References
- Class notes 
- [Salsa20 specification - Daniel J. Bernstein](http://cr.yp.to/snuffle/spec.pdf)
- [D. Boneh and Victor Shoup. A Graduate Course in Applied Cryptography.
Available](https://toc.cryptobook.us/)